<a href="https://colab.research.google.com/github/RaiahMohamedAmine/Movie-Reviews-Classification/blob/master/Film_Reviews_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Importing IMBD REVIEWS Dataset**

In [2]:
import tensorflow_datasets as tfds

dataset,info = tfds.load('imdb_reviews',as_supervised=True,with_info=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2NTMWA/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2NTMWA/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete2NTMWA/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


# **Extracting Train and Test Sentences and Labels**

In [3]:
import numpy as np
test_data,train_data = dataset['test'] ,dataset['train']
train_sentences , train_labels, test_sentences,test_labels = [] , [] , [] , []
for s,l in test_data :
  test_sentences.append(str(s.numpy()))
  test_labels.append(l.numpy())
for s,l in train_data :
  train_sentences.append(str(s.numpy()))
  train_labels.append(l.numpy())

train_labels= np.array(train_labels)
test_labels = np.array(test_labels)
train_sentences=np.array(train_sentences)
test_sentences = np.array(test_sentences)

print(train_sentences.shape)
print(train_labels.shape)
print(test_sentences.shape)
print(test_labels.shape)

#Expected Output : 
#(25000,) 
#(25000,) 
#(25000,) 
#(25000,)

(25000,)
(25000,)
(25000,)
(25000,)


# **Setting Tokenizer**

In [27]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow as tf

vocab_size =10000
max_len=150
post='post'
oov ="<OOV>"
embedding_dim= 16

tokenizer = Tokenizer(num_words = vocab_size,oov_token=oov)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

sequences = tokenizer.texts_to_sequences(train_sentences)
padded = pad_sequences(sequences,maxlen=max_len,padding=post,truncating=post)

test_seq = tokenizer.texts_to_sequences(test_sentences)
test_pad = pad_sequences(test_seq,maxlen=max_len,padding=post,truncating=post)

print(len(padded[0]))
print(len(test_pad[0]))

# Expected ouput : 
#150 
#150

150
150


# **Define Neural Network With Flatten Method**

In [28]:
model = tf.keras.Sequential([
     tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_len),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(6,activation='relu'),
     tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

model.fit(
    padded,
    train_labels,
    epochs=10,
    batch_size=2500,
    validation_data=(test_pad,test_labels)
)

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_11 (Embedding)     (None, 150, 16)           160000    
_________________________________________________________________
flatten_11 (Flatten)         (None, 2400)              0         
_________________________________________________________________
dense_22 (Dense)             (None, 6)                 14406     
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 7         
Total params: 174,413
Trainable params: 174,413
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
10/10 [==============================] - 1s 59ms/step - loss: 0.6924 - accuracy: 0.5109 - val_loss: 0.6914 - val_accuracy: 0.5336
Epoch 2/10
10/10 [==============================] - 1s 50ms/step - loss: 0.6842 - accuracy: 0.6405 - val

# **Define Neural Network With Global Average Pooling Method**

In [30]:
model = tf.keras.Sequential([
     tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=max_len),
     tf.keras.layers.GlobalAveragePooling1D(),
     tf.keras.layers.Dense(6,activation='relu'),
     tf.keras.layers.Dense(1,activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.summary()

model.fit(
    padded,
    train_labels,
    epochs=10,
    batch_size=2500,
    validation_data=(test_pad,test_labels)
)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_13 (Embedding)     (None, 150, 16)           160000    
_________________________________________________________________
global_average_pooling1d_1 ( (None, 16)                0         
_________________________________________________________________
dense_24 (Dense)             (None, 6)                 102       
_________________________________________________________________
dense_25 (Dense)             (None, 1)                 7         
Total params: 160,109
Trainable params: 160,109
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
10/10 [==============================] - 1s 56ms/step - loss: 0.6928 - accuracy: 0.4987 - val_loss: 0.6921 - val_accuracy: 0.5097
Epoch 2/10
10/10 [==============================] - 0s 45ms/step - loss: 0.6911 - accuracy: 0.5213 - val